In [1]:
import sys
import os
from dotenv import load_dotenv

load_dotenv(os.path.expanduser('~/.env'), verbose=True)

data_dir = os.getenv('DATA_IGN_DIR')
adapter_lib_path = os.getenv('ADAPTER_LIB_PATH')

sys.path.insert(0, adapter_lib_path)

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"] = ''
from dataclasses import dataclass, field
from typing import Optional

import datasets
from datasets import load_dataset, concatenate_datasets

import evaluate
import transformers
from transformers import (
    AutoConfig,
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    AutoAdapterModel,
    DataCollatorWithPadding,
    EvalPrediction,
    HfArgumentParser,
    PreTrainedTokenizerFast,
    TrainingArguments,
    default_data_collator,
    set_seed,
    PfeifferConfig
)
from transformers.adapters import AdapterArguments, setup_adapter_training
from transformers.trainer_utils import get_last_checkpoint
from transformers.utils import check_min_version
from transformers.utils.versions import require_version
from utils_qa import postprocess_qa_predictions

import transformers.adapters.composition as ac

from pdb import set_trace
from tqdm import tqdm
import json
from datetime import datetime
import random
import numpy as np

from transformers.adapters.heads import ClassificationHead
from torch.nn import CrossEntropyLoss, MSELoss
from transformers.trainer_utils import EvalLoopOutput
from transformers import EarlyStoppingCallback

import torch.nn as nn
import torch.nn.functional as F
import torch

import math
import time
from pprint import pprint

from transformers import Trainer
from transformers.trainer_utils import PredictionOutput, speed_metrics

from sklearn.metrics import f1_score, accuracy_score
from collections import defaultdict
import shutil

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_count = torch.cuda.device_count()
print(device, device_count)

adapter_info = {
                'bert-base-uncased':
                    {
                        # 'comqa': 'AdapterHub/bert-base-uncased-pf-comqa',
                        # 'cq': 'AdapterHub/bert-base-uncased-pf-cq',
                        # 'drop': 'AdapterHub/bert-base-uncased-pf-drop',
                        # 'duorc_p': 'AdapterHub/bert-base-uncased-pf-duorc_p',
                        # 'duorc_s': 'AdapterHub/bert-base-uncased-pf-duorc_s',
                        'hotpotqa': 'AdapterHub/bert-base-uncased-pf-hotpotqa',
                        'newsqa': 'AdapterHub/bert-base-uncased-pf-newsqa',
                        'quoref': 'AdapterHub/bert-base-uncased-pf-quoref',
                        'squad': 'AdapterHub/bert-base-uncased-pf-squad',
                        'squad_v2': 'AdapterHub/bert-base-uncased-pf-squad_v2',
                        'wikihop': 'AdapterHub/bert-base-uncased-pf-wikihop'
                    },
                'roberta-base':
                    {
                        # 'comqa': 'AdapterHub/roberta-base-pf-comqa',
                        # 'cq': 'AdapterHub/roberta-base-pf-cq',
                        # 'duorc_p': 'AdapterHub/roberta-base-pf-duorc_p',
                        'duorc_s': 'AdapterHub/roberta-base-pf-duorc_s',
                        'hotpotqa': 'AdapterHub/roberta-base-pf-hotpotqa',
                        'newsqa': 'AdapterHub/roberta-base-pf-newsqa',
                        'quoref': 'AdapterHub/roberta-base-pf-quoref',
                        'squad': 'AdapterHub/roberta-base-pf-squad',
                        'squad_v2': 'AdapterHub/roberta-base-pf-squad_v2',
                        'wikihop': 'AdapterHub/roberta-base-pf-wikihop'
                        
                    }
               }

current_time = datetime.now().strftime('%Y%m%d-%H%M%S')

cpu 0


In [3]:
# if len(sys.argv) - 1 != 1:
#     print('Argument error')
#     exit(1)

# _, arg1 = sys.argv

# sample_size = int(arg1)

sample_size = 5000

In [4]:
task_list = ['duorc_s', 'quoref', 'squad', 'squad_v2']

moe_task = 'qa'

task_name_str = f'gating_{moe_task}_sample{sample_size}'
model_name_or_path = 'roberta-base'
max_seq_length = 384
max_answer_length = 30
doc_stride = 128
n_best_size = 20
version_2_with_negative = True
null_score_diff_threshold = 0.0
train_test_rate = 0.2

adapter_k = 2
noisy_gating = True
gating_layer = [0]

num_labels = 2

train_test_ratio = 0.2
random_seed = 0

set_seed(random_seed)
torch.manual_seed(random_seed)
torch.cuda.manual_seed(random_seed)
np.random.seed(random_seed)
random.seed(random_seed)




In [5]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name_or_path,
    use_fast=True,
)

question_column_name = "question"
context_column_name = "context"
answer_column_name = "answers"

def process_data(dataset, eval=False):
    # Padding side determines if we do (question|context) or (context|question).
    pad_on_right = tokenizer.padding_side == "right"
    
    max_seq_len = min(max_seq_length, tokenizer.model_max_length)

    column_names = dataset.column_names

    # Training preprocessing
    def prepare_train_features(examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]
    
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_len,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length",
        )
    
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
        # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples.pop("offset_mapping")
        
        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []
    
        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)
    
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])
    
                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1
    
                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1
    
                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)
    
        return tokenized_examples
    
    # Validation preprocessing
    def prepare_validation_features(examples):
        # Some of the questions have lots of whitespace on the left, which is not useful and will make the
        # truncation of the context fail (the tokenized question will take a lots of space). So we remove that
        # left whitespace
        examples[question_column_name] = [q.lstrip() for q in examples[question_column_name]]
    
        # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
        # in one example possible giving several features when a context is long, each of those features having a
        # context that overlaps a bit the context of the previous feature.
        tokenized_examples = tokenizer(
            examples[question_column_name if pad_on_right else context_column_name],
            examples[context_column_name if pad_on_right else question_column_name],
            truncation="only_second" if pad_on_right else "only_first",
            max_length=max_seq_len,
            stride=doc_stride,
            return_overflowing_tokens=True,
            return_offsets_mapping=True,
            padding="max_length"
        )
    
        # Since one example might give us several features if it has a long context, we need a map from a feature to
        # its corresponding example. This key gives us just that.
        sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
                # The offset mappings will give us a map from token to character position in the original context. This will
        # help us compute the start_positions and end_positions.
        offset_mapping = tokenized_examples["offset_mapping"]

        tokenized_examples["dataset_ids"] = []
        for i in range(len(tokenized_examples["input_ids"])):
            # This gets the dataset_id of the original example each feature was created from.
            sample_index = sample_mapping[i]
            tokenized_examples["dataset_ids"].append(examples["dataset_ids"][sample_index])
    
        # Let's label those examples!
        tokenized_examples["start_positions"] = []
        tokenized_examples["end_positions"] = []
    
        for i, offsets in enumerate(offset_mapping):
            # We will label impossible answers with the index of the CLS token.
            input_ids = tokenized_examples["input_ids"][i]
            cls_index = input_ids.index(tokenizer.cls_token_id)
    
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            answers = examples[answer_column_name][sample_index]
            # If no answers are given, set the cls_index as answer.
            if len(answers["answer_start"]) == 0:
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Start/end character index of the answer in the text.
                start_char = answers["answer_start"][0]
                end_char = start_char + len(answers["text"][0])
    
                # Start token index of the current span in the text.
                token_start_index = 0
                while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                    token_start_index += 1
    
                # End token index of the current span in the text.
                token_end_index = len(input_ids) - 1
                while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                    token_end_index -= 1
    
                # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
                if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                    tokenized_examples["start_positions"].append(cls_index)
                    tokenized_examples["end_positions"].append(cls_index)
                else:
                    # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                    # Note: we could go after the last offset if the answer is the last word (edge case).
                    while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                        token_start_index += 1
                    tokenized_examples["start_positions"].append(token_start_index - 1)
                    while offsets[token_end_index][1] >= end_char:
                        token_end_index -= 1
                    tokenized_examples["end_positions"].append(token_end_index + 1)
    
    
        # For evaluation, we will need to convert our predictions to substrings of the context, so we keep the
        # corresponding example_id and we will store the offset mappings.
        tokenized_examples["example_id"] = []
    
        for i in range(len(tokenized_examples["input_ids"])):
            # Grab the sequence corresponding to that example (to know what is the context and what is the question).
            sequence_ids = tokenized_examples.sequence_ids(i)
            context_index = 1 if pad_on_right else 0
    
            # One example can give several spans, this is the index of the example containing this span of text.
            sample_index = sample_mapping[i]
            tokenized_examples["example_id"].append(examples["id"][sample_index])
    
            # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
            # position is part of the context or not.
            tokenized_examples["offset_mapping"][i] = [
                (o if sequence_ids[k] == context_index else None)
                for k, o in enumerate(tokenized_examples["offset_mapping"][i])
            ]
    
        return tokenized_examples

    if eval:
        column_names.remove('dataset_ids')
        eval_examples = dataset
        # Validation Feature Creation
        eval_dataset = eval_examples.map(
            prepare_validation_features,
            batched=True,
            remove_columns=column_names,
            desc="Running tokenizer on evaluation dataset",
        )
        return eval_dataset, eval_examples
    else:
        # Create train feature from dataset
        train_dataset = dataset.map(
            prepare_train_features,
            batched=True,
            remove_columns=column_names,
            desc="Running tokenizer on train dataset",
        )
        return train_dataset

In [6]:
raw_datasets_list = []
for task_name in task_list:
    train_data_path = os.path.join(data_dir, f'data_qa/{task_name}/{task_name}_train.json')
    dev_data_path = os.path.join(data_dir, f'data_qa/{task_name}/{task_name}_dev.json')
    
    raw_datasets = load_dataset('json', data_files={'train': train_data_path, 'validation': dev_data_path})
    raw_datasets_list.append(raw_datasets)

In [7]:
def sample_dataset(dataset, sample_size, random_seed=None):
    def set_sample_id(example, idx, new_ids):
        sample_id = new_ids[idx]
        example['id'] = example['id'] + f'-{sample_id}'
        return example
    # If the sample size is smaller than or equal to the dataset, shuffle and select
    if sample_size <= len(dataset):
        shuffled_dataset = dataset.shuffle(seed=random_seed)
        sampled_dataset = shuffled_dataset.select(range(sample_size))
    # If the sample size is larger, resample with replacement and assign new IDs
    else:
        indices = [random.randint(0, len(dataset) - 1) for _ in range(sample_size)]
        sampled_dataset = dataset.select(indices)

    # Assign new unique IDs to each entry in the oversampled dataset
    new_ids = range(len(sampled_dataset))
    sampled_dataset = sampled_dataset.map(set_sample_id, with_indices=True, fn_kwargs={'new_ids': new_ids})

    return sampled_dataset

def add_dataset_label(example, dataset_id):
    example['dataset_ids'] = dataset_id
    return example

In [13]:
for i, _dataset in enumerate(raw_datasets_list):
    for k, dataset in _dataset.items():
        raw_datasets_list[i][k] = dataset.map(add_dataset_label, fn_kwargs={'dataset_id': i})

_dataset_list = [dataset['train'].train_test_split(test_size=train_test_ratio, shuffle=True, seed=random_seed) for dataset in raw_datasets_list]

train_dataset_list = [process_data(dataset['train'], eval=True) for dataset in _dataset_list]
# valid_dataset_list = [process_data(dataset['test'], eval=True) for dataset in _dataset_list]

# eval_dataset_list = [process_data(dataset['validation'], eval=True) for dataset in raw_datasets_list]

Running tokenizer on evaluation dataset:   0%|          | 0/33915 [00:00<?, ? examples/s]

Running tokenizer on evaluation dataset:   0%|          | 0/15506 [00:00<?, ? examples/s]

Running tokenizer on evaluation dataset:   0%|          | 0/70079 [00:00<?, ? examples/s]

Running tokenizer on evaluation dataset:   0%|          | 0/104255 [00:00<?, ? examples/s]

In [14]:
[len(d)/len(e)for d, e in train_dataset_list]

[(Dataset({
      features: ['dataset_ids', 'input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_id'],
      num_rows: 125837
  }),
  Dataset({
      features: ['context', 'answers', 'title', 'id', 'question', 'dataset_ids'],
      num_rows: 33915
  })),
 (Dataset({
      features: ['dataset_ids', 'input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_id'],
      num_rows: 30940
  }),
  Dataset({
      features: ['id', 'answers', 'question', 'title', 'context', 'dataset_ids'],
      num_rows: 15506
  })),
 (Dataset({
      features: ['dataset_ids', 'input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'end_positions', 'example_id'],
      num_rows: 71460
  }),
  Dataset({
      features: ['context', 'id', 'answers', 'title', 'question', 'dataset_ids'],
      num_rows: 70079
  })),
 (Dataset({
      features: ['dataset_ids', 'input_ids', 'attention_mask', 'offset_mapping', 'start_positions', 'e

In [16]:
[len(d)/len(e)for d, e in train_dataset_list]

[3.7103641456582634, 1.9953566361408488, 1.0197063314259622, 1.019682509232171]